### Table of contents
1. [Build DF](#top)
2. [Add chapter & verse number](#chapter)
3. [Openers!](#opener)
4. [Arabic text!](#text)
5. [Length of Sura!](#length)

<a id="top"> </a>
### Build the data frame and store it using inline magic

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# read the Quran text file
file = open('quran_cs.txt', 'r', encoding='utf-8')
qList = file.readlines()
file.close()

In [ ]:
qList[7], len(qList)

In [ ]:
#  read Quran as a single string
file = open('quran_cs.txt', 'r', encoding='utf-8')
qStr = file.read().replace('\n', ' ')
file.close()

In [ ]:
len(qStr), qStr.count(' '), qStr[:150]

#### Columns 'Chapter' and 'Verse'
<a id="chapter"> </a>

In [ ]:
# Retrieve chapters and verses numbers
i=0
sura= np.arange(len(qList))
sura[0]=1

aya= np.arange(len(qList))

# get the sura number
for vr in qList:
    if (i): 
        sura[i]= int(vr.split(chr(124),1)[0])
    aya[i]=  int(vr.split(chr(124),2)[1])
    i+=1
    
    
# this gives how many occurences of the number in the list (which is the length of the specific sura- number of ayats)    
list(sura).count(2)

##### column sura name

In [ ]:
# read the Quran text file
file = open('surat_names.txt', 'r', encoding='utf-8')
namesList = file.readlines()
file.close()

In [ ]:
namesList[0].split()[0]
# len(namesList)

In [ ]:
namesList[sura[7]-1].split()[0]

In [ ]:
suratNames = []

for i in range(6236):
    suratNames.append(namesList[sura[i]-1].split()[0])

In [ ]:
# suratNames

##### column openers
<a id="opener"> </a>

In [ ]:
# read the Quran text file
file = open('ChapStart.txt', 'r', encoding='utf-8')
openList = file.readlines()
file.close()

In [ ]:
openList[0].split()[0]
# len(openList)



In [ ]:
# create column for openers

openers = [' '] * 6236
indx = 0

for i in range(6236):
    if (int(sura[i]) == int(openList[indx].split()[0])):
        openers[i] = openList[indx].split()[1]
        print(sura[i], openList[indx].split()[1])
        indx += 1
        if (indx == len(openList)): break  # reach end of list of surat with openers 

                

In [ ]:
# openers

##### column text
<a id="text"> </a>

In [ ]:

# After retreiving chapter & verse numbers, we remove numbers from quran text.
dict = {'|':'', '0': '', '1': '', '2': '', '3':'', '4': '', '5': '', '6': '', '7':'', '8': '', '9': '', '\n':'', '\ufeff':''}
trantab = str.maketrans(dict)
i=0
qText = qList
for vr in qList:
    qText[i] = vr.translate(trantab)
    i+=1
    
# qText

In [ ]:
qText[7]

##### Column Nbr of Verses in chapter
<a id="length"> </a>

In [ ]:
# calculate the length of chapters
slen =  np.arange(114)
temp = 1
ln = 0
j=0
for i in range(6234):
    if ( sura[i] == temp):
        ln += 1
    else:
#         print(ln)
        slen[j] = ln
        j += 1
        ln = 1
        temp += 1
        
slen[113] = 6

# Create a column for chapters lengths
verses = np.arange(6236)
for i in range(6236):
    verses[i] = slen[sura[i]-1]
        
    


##### columns letters and words count

In [ ]:
# add length of each verse by words and letters
sletters = np.arange(6236)
swords = np.arange(6236)

for i in range(6236):
    sletters[i] = len(qList[i])
    swords[i] = len(qList[i].split())

** Column qDigit skip space. transform letters to numbers. **
**- Space is removed**

In [ ]:
# convert quran letters to numbers 
# qDigit_no_sp is a list of 6236 lists, where each element is a list of numbers representing  verse letters (excluding space)

qDigit_no_sp= [[] for x in range(6236)]
j=0

while (j < 6236):
    st = qList[j]
#     print(st)
    for i in range(0,int(len(st))):
        if ord(st[i]) == 32:
            continue
        else:
            qDigit_no_sp[j].append(ord(st[i]) - 1568)  #ascii code of arabic letters starts at 1568 (there is a gap from 1594 to 1601)
    j+=1


c=np.asarray(qDigit_no_sp[1])


**- Space is not removed**

In [ ]:
# convert quran letters to numbers 
# qDigit_sp is a list of 6236 lists, where each element is a list of numbers representing  verse letters (excluding space)

qDigit_sp= [[] for x in range(6236)]
j=0

while (j < 6236):
    st = qList[j]
#     print(st)
    for i in range(0,int(len(st))):
        if ord(st[i]) == 32:
            qDigit_sp[j].append(50)
        else:
            qDigit_sp[j].append(ord(st[i]) - 1568)  #ascii code of arabic letters starts at 1568 (there is a gap from 1594 to 1601)
    j+=1


c=np.asarray(qDigit_sp[0])

** Merge all hamza letters and the 2 types of taa**

In [ ]:
# merge all Alif types and Taa types
import copy
qDigit_merged = copy.deepcopy(qDigit_no_sp)

for v in range(len(qDigit_merged)):
    for l in range(len(qDigit_merged[v])):
        if qDigit_merged[v][l] in (2,3,4,5,6,7,41):
            qDigit_merged[v][l] = 1
        if qDigit_merged[v][l] == 10:
            qDigit_merged[v][l] = 9 
            

In [ ]:
i = 10
print(qDigit_no_sp[i])
print(qDigit_no_sp[i])
print(qDigit_no_sp[i])

** alif maqsoura is not hamza **

In [ ]:
# merge all Alif types and Taa types 
# Alif maqsoura digit=41 is not included in all Alif types
import copy
qDigit_maq = copy.deepcopy(qDigit_no_sp)

for v in range(len(qDigit_maq)):
    for l in range(len(qDigit_maq[v])):
        if qDigit_maq[v][l] in (2,3,4,5,6,7):  # 41 is remomved from list to not merge alif maqsoura
            qDigit_maq[v][l] = 1
        if qDigit_maq[v][l] == 10:
            qDigit_maq[v][l] = 9 

###### Transform arabic letters to  abjad numerals.      Germatic code

In [ ]:
# read the file with codes

# use strip to get rid off \n
codes = [line.strip() for line in open("ord_to_code.txt", 'r', encoding='utf-8-sig' )]


for i in range(len(codes)):
    codes[i] = codes[i].split()    # split the list rows to 2 columns
    codes[i][0] = int(codes[i][0]) # convert from string to integer
    codes[i][1] = int(codes[i][1]) # convert from string to integer
    
# print(codes)




In [ ]:
for i in range(len(codes)):
    print(codes[i][0], codes[i][1], chr(codes[i][0]))   # chr transform code to arabic letter.
    

In [ ]:
# create a dictionary with key= first culumn (ascii code), value = second culumn (germatic code) 
dictCodes = {}
for i in range(len(codes)):
    dictCodes[codes[i][0]] = codes[i][1]
    
print (dictCodes)

In [ ]:
# convert quran letters to germatic codes 
# qDigit is a list of 6236 lists, where each element is a list of numbers representing  verse letters (excluding space)

qDigit_ger= [[] for x in range(6236)]
j=0

while (j < 6236):
    st = qList[j]
#     print(st)
    for i in range(0,int(len(st))):
        if ord(st[i]) == 32:
            continue
        else:
            qDigit_ger[j].append(dictCodes[ord(st[i])])  #ascii code of arabic letters starts at 1568 (there is a gap from 1594 to 1601)
    j+=1


c2=np.asarray(qDigit_ger[0])

##### Add column, split verse to a list of letters

In [ ]:
# Convert veres to list of arabic letters

qletters= [[] for x in range(6236)]
j=0

while (j < 6236):
    st = qList[j]
    for i in range(0,int(len(st))):
        if ord(st[i]) == 32:
#             continue
            qletters[j].append('_')
        else:
            qletters[j].append(st[i])
    j+=1


c=np.asarray(qletters[0])



##### Add column as Q transliteration

In [ ]:
# use strip to get rid off \n
lettersCodes = [line.strip() for line in open("arabic_latin.txt", 'r', encoding='utf-8-sig' )]
#print(LetterCode)

# split the list rows to 2 columns
for i in range(len(lettersCodes)):
    lettersCodes[i] = lettersCodes[i].split()

#convert first column to integer    
for i in range(len(lettersCodes)):
    lettersCodes[i][0] = int(lettersCodes[i][0])
    
# print(lettersCodes)
dTransliter = {t[0]:t[1:] for t in lettersCodes}
dTransliter[32] = ' '
dTransliter

In [ ]:
for i in range(len(codes)):
    print(codes[i][0], dTransliter[codes[i][0]], codes[i][1], chr(codes[i][0]))   # chr transform code to arabic letter.
#     print(dTransliter[codes[i][0]])




In [ ]:
# convert quran letters to numbers 
# qDigit_sp is a list of 6236 lists, where each element is a list of numbers representing  verse letters (excluding space)

transText= []
tmp = ''
ind = 0

for st in qText:
#     print(st)
    for i in range(len(st)):
#         print(ord(st[i]), dTransliter[ord(st[i])][0] )
        tmp += dTransliter[ord(st[i])][0]      
#     print(tmp)
    transText.append(tmp)
    tmp = ''
    


In [ ]:
len(transText)

In [ ]:
(openers)

**- Add column, keep only openers letters on surat with openers**

In [ ]:
# loop on verse
# check sura opener, if not space
#   then substitute verse with verse of openers
import re   

In [ ]:
idx = 0
bOpen = False

openText = []

for vr in transText:
    if (openers[idx] != ' '):
        bOpen = True
        tmp = sura[idx]
        if (openers[idx] == '7m-3s9'): # problem at hm-3sq, it includes hyphen '-'
            openers[idx] = '7m'
        openSubset  = "[^" +  openers[idx] + " ]"  # keep space use " ]", or "]" to remove spcae also
#         print(openSubset)

        
    if (bOpen & (sura[idx] == tmp)):
        vr = re.sub(openSubset, '', vr)
#         print(vr)  
#         print(idx, 'is ON on sura',sura[idx] )
    
    openText.append(vr)    
        
    idx += 1
    
#     if (idx == 1000):
#         break

In [ ]:
# openText[0:600]

### Create the dataframe

In [ ]:
qdf = pd.DataFrame({'sura':sura, 'Name':suratNames, 'opener': openers, 'len':verses, 'nbr_letters':sletters, 
                    'nbr_words':swords,  
                    'Letters':qletters, 'Aya':aya, 'Aya_code':qDigit_ger,'Aya_digit1':qDigit_no_sp,'Aya_digit3':qDigit_sp,
                    'Aya_digit_alif':qDigit_merged,'Aya_digit_maqsra':qDigit_maq,'text': qText, 'transText':transText,
                    'openText': openText })


In [ ]:
qdf.head()

In [ ]:
%store qdf

In [ ]:
tText = list(qdf['openText'][qdf['sura'] ==40])
tText

In [ ]:
text = list(qdf['text'][qdf['sura'] ==40])
text

In [ ]:
import sys
sys.version_info.major

In [ ]:
qdf.dtypes

In [ ]:
qdf.to_csv("qdf.csv", sep=',', encoding='utf-8')

##### using qGrid

In [ ]:
import qgrid



In [68]:
randn = np.random.randn
df_types = pd.DataFrame({
    'A' : pd.Series(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08', '2013-01-09'],
                    index=list(range(9)),dtype='datetime64[ns]'),
    'B' : pd.Series(randn(9),index=list(range(9)),dtype='float32'),
    'C' : pd.Categorical(["washington", "adams", "washington", "madison", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'D' : ["foo", "bar", "buzz", "bippity","boppity", "foo", "foo", "bar", "zoo"] })

df_types['E'] = df_types['D'] == 'foo'

qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget


A Jupyter Widget

In [ ]:
qgrid.show_grid

In [ ]:
import ipywidgets as ipyw


In [ ]:
# randn = np.random.randn
# df_types = pd.DataFrame(np.random.randint(1,14,14))
# qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
# qgrid_widget.layout = ipyw.Layout(width='50%')
# qgrid_widget

In [69]:
from IPython.display import display, HTML

# Assuming that dataframes df1 and df2 are already defined:
# print ()"Dataframe 1:")
display(qgrid_widget)
# print () "Dataframe 2:")
display(HTML(qdf.to_html()))

A Jupyter Widget

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [70]:
pd.options.display.html.table_schema = True

In [87]:
qdf.style

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [82]:
from collections import OrderedDict, Counter
# Remembers the order the keys are added!
x = OrderedDict(a=5, d=1, b=3)
# Counts the frequency of each character
y = Counter("Hello World!!!")

x,y

(OrderedDict([('a', 5), ('d', 1), ('b', 3)]),
 Counter({' ': 1,
          '!': 3,
          'H': 1,
          'W': 1,
          'd': 1,
          'e': 1,
          'l': 3,
          'o': 2,
          'r': 1}))

<table>
  <tr>
    <th>first name</th>
    <th>last name</th>
  </tr>
  <tr>
    <td>Med</td>
    <td>GHmari</td>
  </tr>
  <tr>
    <td>Abdou</td>
    <td>Rhmiro</td>
  </tr>
</table>

In [94]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

A Jupyter Widget

In [95]:
from IPython.display import display,HTML
#ipywidgets imports
from __future__ import print_function
from ipywidgets import *
import ipywidgets as widgets

#EDIT
#added an html widget for output
w3 = widgets.HTML(value="")

def my_function(a, b):
    html_table = '<table><thead>'
    for col in a:
        html_table += '<th>'+col+'</th>'
    html_table += '</thead><tbody>'
    for row in b:
        html_table += '<tr>'
        for col in a:
            html_table += '<td>'+row+'</td>'
        html_table += '</tr>'
    html_table += '</tbody></table>'

    #EDIT
    w3.value = html_table
    #return HTML(html_table)
    #return html_table

widget1 = widgets.SelectMultiple(
    description="columns:",
    options=['A', 'B', 'C', 'D', 'E', 'F', 'G']
)

widget2 = widgets.SelectMultiple(
    description="rows:",
    options=['A', 'B', 'C', 'D', 'E', 'F', 'G']
)

#EDIT
interact(my_function, a=widget1, b=widget2)
display(w3)

A Jupyter Widget

A Jupyter Widget

In [97]:
# from IPython.html import widgets 
from IPython.display import display 

out = widgets.Output() 
display(out)

# print("prints to output area")

A Jupyter Widget

prints to output area


In [ ]:
from IPython.core.display import display, HTML
display(HTML(open('style.css', 'r').read()));
